In [99]:
import os
import mysql.connector
import pandas as pd
import numpy as npy
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline


In [107]:
connection = mysql.connector.connect(host = 'localhost',
                                    user = 'root',
                                    passwd = '!'
                                    db = 'first')

SyntaxError: invalid syntax (3290934807.py, line 4)

In [104]:
connection

In [106]:
baseball_tables = pd.read_sql_query('use first', connection)
baseball_tables = pd.read_sql_query('SHOW TABLES FROM first', connection)

DatabaseError: Execution failed on sql 'use first': 1049 (42000): Unknown database 'first'

In [16]:
baseball_tables

,Tables_in_first
0,baseball_players
1,mlb_teams


In [18]:
tables = baseball_tables['Tables_in_first']

In [20]:
for table_name in tables:
    output = pd.read_sql_query('DESCRIBE {}'.format(table_name), connection)
    print(table_name)
    print(output, '\n')

baseball_players
        Field              Type Null Key Default Extra
0   full_name   b'varchar(255)'  YES        None      
1        team     b'varchar(4)'  YES        None      
2       games            b'int'  YES        None      
3     at_bats            b'int'  YES        None      
4        runs            b'int'  YES        None      
5        hits            b'int'  YES        None      
6     doubles            b'int'  YES        None      
7     triples            b'int'  YES        None      
8      Homers            b'int'  YES        None      
9        RBIs            b'int'  YES        None      
10         BP  b'decimal(20,4)'  YES        None      
11    singles            b'int'  YES        None       

mlb_teams
            Field            Type Null Key Default Extra
0         team_cd   b'varchar(5)'  YES        None      
1  team_full_name  b'varchar(50)'  YES        None       



/Users/seankarkhanis/opt/anaconda3/lib/python3.9/site-packages/pandas/io/sql.py:761: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(
/Users/seankarkhanis/opt/anaconda3/lib/python3.9/site-packages/pandas/io/sql.py:761: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(


## Below are a series of Business Questions and Prompts based on MLB Data from the year 2019. I have written SQL queries to make sense of this data and solve these questions / prompts. 

## I have used the following format to display my findings:



## Business Question / Prompt
## Readable SQL Query Code in text cell
## SQL code that is run to get output
## Output

## What players played the most games in the season?

In [76]:
x = pd.read_sql_query('select full_name, games from baseball_players order by games desc', connection)


/Users/seankarkhanis/opt/anaconda3/lib/python3.9/site-packages/pandas/io/sql.py:761: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(


## The output of this query is below:

In [102]:
x

,full_name,games
0,Starlin Castro,162
1,Whit Merrifield,162
2,Marcus Semien,162
3,Jorge Soler,162
4,Jonathan Villar,162
...,...,...
818,Matthew Boyd,1
819,Zack Greinke,1
820,David Price,1
821,Gregory Soto,1


## Give the players and their hitting stats of those that have hit more Home Runs than league average.

SELECT full_name, team, games, at_bats, hits, Homers
    FROM baseball_players
    WHERE homers > (SELECT avg(Homers) 
    FROM baseball_players) 
order by Homers desc;

In [56]:
x = pd.read_sql_query('SELECT full_name, team, games, at_bats, hits, Homers FROM baseball_players WHERE homers > (SELECT avg(Homers)  FROM baseball_players)  order by Homers desc;', 
                      connection)


/Users/seankarkhanis/opt/anaconda3/lib/python3.9/site-packages/pandas/io/sql.py:761: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(


## The output of this query is below:

In [57]:
x

,full_name,team,games,at_bats,hits,Homers
0,Pete Alonso,NYM,161,597,155,53
1,Eugenio Suarez,CIN,159,575,156,49
2,Jorge Soler,KCR,162,589,156,48
3,Cody Bellinger,LAD,156,558,170,47
4,Mike Trout,LAA,134,470,137,45
...,...,...,...,...,...,...
283,Adalberto Mondesi,KCR,102,415,109,9
284,Dylan Moore,SEA,113,247,51,9
285,Addison Russell,CHC,82,215,51,9
286,Raimel Tapia,COL,138,426,117,9


## Looking up a team's full name using inner join

select  t.team_full_name, b.full_name, b.games as games_played
from baseball_players as b
INNER JOIN mlb_teams as t on (b.team = t.team_cd)
order by t.team_full_name, b.full_name;

In [62]:
x = pd.read_sql_query('select  t.team_full_name, b.full_name, b.games as games_played from baseball_players as b INNER JOIN mlb_teams as t on (b.team = t.team_cd) order by t.team_full_name, b.full_name', connection)


/Users/seankarkhanis/opt/anaconda3/lib/python3.9/site-packages/pandas/io/sql.py:761: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(


## The output of this query is below:

In [63]:
x

,team_full_name,full_name,games_played
0,Arizona Diamondbacks,Abraham Almonte,17
1,Arizona Diamondbacks,Adam Jones,137
2,Arizona Diamondbacks,Alex Avila,63
3,Arizona Diamondbacks,Alex Young,16
4,Arizona Diamondbacks,Blake Swihart,31
...,...,...,...
818,Washington Nationals,Stephen Strasburg,33
819,Washington Nationals,Trea Turner,122
820,Washington Nationals,Victor Robles,155
821,Washington Nationals,Wilmer Difo,43


## What players have a batting percentage above league average while having at least 100 at bats?

select team, full_name, games, BP, at_bats, hits
from baseball_players
WHERE BP > (SELECT avg(BP) FROM baseball_players) AND at_bats > 100
order by 4 DESC;

In [65]:
x = pd.read_sql_query('select team, full_name, games, BP, at_bats, hits from baseball_players WHERE BP > (SELECT avg(BP) FROM baseball_players) AND at_bats > 100 order by 4 DESC;', connection)


/Users/seankarkhanis/opt/anaconda3/lib/python3.9/site-packages/pandas/io/sql.py:761: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(


## The output of this query is below:

In [66]:
x

,team,full_name,games,BP,at_bats,hits
0,WSN,Howie Kendrick,121,0.3443,334,115
1,CHW,Tim Anderson,123,0.3353,498,167
2,MIN,Luis Arraez,92,0.3344,326,109
3,SFG,Donovan Solano,81,0.3302,215,71
4,MIL,Christian Yelich,130,0.3292,489,161
...,...,...,...,...,...,...
316,STL,Paul DeJong,159,0.2333,583,136
317,DET,Christin Stewart,104,0.2331,369,86
318,HOU,Jake Marisnick,120,0.2329,292,68
319,ATL,Johan Camargo,98,0.2328,232,54


## What player leads their team in games played? (Use hits to break ties)

select * from 
(
	Select team, games, full_name,
	Rank()
	OVER(
	PARTITION BY team
	Order by games DESC, hits DESC) 
	as RNK
	from baseball_players) as myrnks
where rnk = 1;

In [68]:
x = pd.read_sql_query('select * from (Select team, games, full_name, Rank() OVER(PARTITION BY team Order by games DESC, hits DESC) as RNK from baseball_players) as myrnks where rnk = 1;', connection)


/Users/seankarkhanis/opt/anaconda3/lib/python3.9/site-packages/pandas/io/sql.py:761: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(


## The output of this query is below:

In [69]:
x

,team,games,full_name,RNK
0,ARI,158,Eduardo Escobar,1
1,ATL,160,Ozzie Albies,1
2,BAL,162,Jonathan Villar,1
3,BOS,156,Rafael Devers,1
4,CHC,155,Kyle Schwarber,1
5,CHW,159,José Abreu,1
6,CIN,159,Eugenio Suarez,1
7,CLE,158,Carlos Santana,1
8,COL,155,Nolan Arenado,1
9,DET,136,Miguel Cabrera,1


 ## Order the MLB teams by team overall batting percentage

Select * from
(
	Select Distinct team, 
	avg(BP)
	OVER(
	partition by team) 
    AS avg_team_BP
	from baseball_players)
    as myRNKs
ORDER BY avg_team_BP DESC;

In [72]:
x = pd.read_sql_query('Select * from(Select Distinct team, avg(BP) OVER(partition by team) AS avg_team_BP from baseball_players) as myRNKs ORDER BY avg_team_BP DESC', connection)


/Users/seankarkhanis/opt/anaconda3/lib/python3.9/site-packages/pandas/io/sql.py:761: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(


## The output of this query is below:

In [73]:
x

,team,avg_team_BP
0,DET,0.267521
1,BAL,0.263061
2,TBR,0.262489
3,HOU,0.261968
4,STL,0.258156
5,MIN,0.254080
6,PIT,0.253970
7,NYM,0.251028
8,TEX,0.250945
9,NYY,0.245921


## Returns the percentage of how much a player contributed to their team in terms of runs scored

In [ ]:
Select team, full_name,runs,
sum(runs) over (partition by team order by team) as team_score,
(runs/sum(runs) over (partition by team order by team))*100 as player_contri_pct
from baseball_players
order by team, runs desc;

In [74]:
x = pd.read_sql_query('select team, full_name,runs, sum(runs) over (partition by team order by team) as team_score, (runs/sum(runs) over (partition by team order by team))*100 as player_contri_pct from baseball_players order by team, runs desc', connection)


/Users/seankarkhanis/opt/anaconda3/lib/python3.9/site-packages/pandas/io/sql.py:761: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(


## The output of this query is below:

,team,full_name,runs,team_score,player_contri_pct
0,ARI,Ketel Marte,97,813.0,11.9311
1,ARI,Eduardo Escobar,94,813.0,11.5621
2,ARI,Christian Walker,86,813.0,10.5781
3,ARI,Nick Ahmed,79,813.0,9.7171
4,ARI,Adam Jones,66,813.0,8.1181
...,...,...,...,...,...
818,WSN,Jake Noll,1,873.0,0.1145
819,WSN,Joe Ross,1,873.0,0.1145
820,WSN,Anibal Sanchez,1,873.0,0.1145
821,WSN,Raudy Read,0,873.0,0.0000
